In [33]:
import cv2
import os
import subprocess
import numpy as np
import pandas as pd
import json

In [34]:
def get_video_properties(video_path):
    
    """
    The function retrieves properties of a video file given its path.

    Parameters:

    video_path (str): The file path to the video.
    
    Returns:

    width (int): The width of the video in pixels.
    height (int): The height of the video in pixels.
    fps (float): The frames per second of the video.
    pix_fmt(str): The pixel format of the video.
    
    """
    
    
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the video width and height
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Get the frames per second (FPS)
    fps = video.get(cv2.CAP_PROP_FPS)

    # Release the video file
    video.release()
    
    ffprobe_cmd = f'ffprobe -v error -select_streams v:0 -show_entries stream=pix_fmt -of json "{video_path}"'

    # Execute the ffprobe command
    result = subprocess.run(ffprobe_cmd, capture_output=True, text=True, shell=True)

    # Parse the JSON output
    metadata = json.loads(result.stdout)
    pix_fmt = metadata['streams'][0]['pix_fmt']

    return width, height, fps, pix_fmt

In [70]:
def createMetadataCsv(directory):
    
    # Create the pandas DataFrame
    df = pd.DataFrame()
    
    filenames = []
    width = []
    height = []
    fps = []
    pixel_format = []

    # iterate over files in the directory
    
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            if '.mp4' in f:
                name = os.path.basename(f)
                name = os.path.splitext(name)[0]

                filenames.append(name)
              
                properties = get_video_properties(f)
                width.append(properties[0])
                height.append(properties[1])
                fps.append(properties[2])
                pixel_format.append(properties[3])

    df = df.assign(Name=filenames,
                   width=width,
                   height=height,                 
                   fps=fps,
                   pixel_format=pixel_format)
    
    #save file
    # Specify the file path and name for the CSV file
    csv_file_path = f"{directory}/metadata.csv"

    # Save the DataFrame to a CSV file
    df.to_csv(csv_file_path, index=False)
    
    return df



In [71]:
#test
createMetadataCsv("/Users/anastasiya/Documents/MASTER/SOSE23/Deep_Encode/last")

,Name,width,height,fps,pixel_format
0,factory_1080p30-Scene-017,1920,1080,30.0,yuv420p
